In [1]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

In [2]:
import minsearch

/workspaces/LLM-_Zoomcamp_CaroVergara/LLM_01_intro/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [3]:
import json

In [4]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [9]:
q = 'the course has already started, can I still enroll?'

In [10]:
index.fit(documents)

In [11]:
boost = {'question':3.0, 'section': 0.5}

results = index.search(
    query = q,
    boost_dict = boost,
    num_results = 5
)


In [12]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [15]:
from openai import OpenAI

In [16]:
client = OpenAI()

In [32]:
def search(query):
    boost = {'question':3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict = {'course':'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 10
    )

    return results

In [48]:
def build_prompt(query,search_results):
    prompt_template = """
You´re a course teaching assitant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question = query, context=context).strip()

    return prompt
    

In [49]:
def llm(prompt):
    response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role":"user", "content":prompt}]
)

    return response.choices[0].message.content

In [50]:
#query = 'the course already started. Can I still enroll?'
query = 'how i can run kafka?'
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)

In [51]:
answer

"To run Kafka, follow the solution provided for a similar issue in your course context related to Python Kafka. You should create a virtual environment and install the necessary packages included in the `requirements.txt` file within that environment. Here's a step-by-step guide:\n\n1. **Create a Virtual Environment:** This step is performed only once.\n   ```bash\n   python -m venv env\n   ```\n\n2. **Activate the Virtual Environment:** You'll need to do this every time you wish to use the virtual environment.\n   - For MacOS/Linux:\n     ```bash\n     source env/bin/activate\n     ```\n   - For Windows:\n     ```bash\n     env\\Scripts\\activate\n     ```\n\n3. **Install Required Packages:**\n   ```bash\n   pip install -r ../requirements.txt\n   ```\n\n4. **Deactivate the Virtual Environment:** When you're done, you can deactivate the environment with:\n   ```bash\n   deactivate\n   ```\n\nAdditionally, ensure that all Docker images are up and running if your setup requires Docker. T

In [52]:
query = 'how i can run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [53]:
rag(query)

"To run Kafka, you should first set up the necessary environment, which may involve creating a virtual environment for your Python projects. Here are the steps:\n\n1. **Create a Virtual Environment** (to run Python with Kafka):\n   - Create the environment: `python -m venv env`\n   - Activate it (every time you need to use it):\n     - On MacOS/Linux: `source env/bin/activate`\n     - On Windows: `env\\Scripts\\activate`\n   - Install the required packages: `pip install -r ../requirements.txt`\n\n2. **Running Java-based Kafka applications**:\n   - Navigate to your project directory.\n   - Run your Java applications using:\n     ```shell\n     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n     ```\n\nThese steps should help you successfully run Kafka-related tasks in your environment depending on whether you're utilizing Python or Java. Note that Docker images should be up and running if you're leveraging them alongside Python scripts."

In [54]:
rag('the course already started. Can I still enroll?')

"Yes, you can still enroll in the course even after it has started. However, be aware that there will be deadlines for submitting the final projects, so it's important not to leave everything for the last minute."